In [52]:
import pandas as pd
# pandas importieren

import zipfile
# zipfile importieren, um zip-Datein nutzen zu können

import csv
# csv importieren, um csv einzulesen

import tempfile
# zur Ablage von tempfiles

from pathlib import Path
# zur Erzeugung des lokalen Verzeichnisses zur Speicherung der tempfiles

import numpy as np
# numpy Library importieren

In [53]:
subdir = Path(tempfile.gettempdir()).joinpath('Journal_Report_VZ')
subdir.mkdir(exist_ok=True, parents=True)
# lokales Verzeichnis "Journal_Report_VZ" (= subdir) erzeugen

In [54]:
from zipfile import ZipFile
test_file_name = r"C:\Users\tinag\Desktop\COUNTER_Journal_Report.zip"
# %% hier lokalen Dateipfad einsetzen %%

with ZipFile(test_file_name, 'r') as zip:
    zip.printdir()
    zip.extractall(subdir)
# zipfile auslesen
# Datein temporär speichern

File Name                                             Modified             Size
counter5_tr_j1_2021-01-2021-12_2000150893.csv  2022-07-01 13:51:14       311258
TR_J1_20220701_0254.csv                        2022-07-15 13:34:44        83493


In [55]:
print(subdir)
#Anzeige erstelltes Verzeichnis subdir

C:\Users\tinag\AppData\Local\Temp\Journal_Report_VZ


In [56]:
for p in subdir.glob('*.csv'):
    print(p)
#Anzeige aller csv-Datein im erstellten Verzeichnis subdir

C:\Users\tinag\AppData\Local\Temp\Journal_Report_VZ\counter5_tr_j1_2021-01-2021-12_2000150893.csv
C:\Users\tinag\AppData\Local\Temp\Journal_Report_VZ\TR_J1_20220701_0254.csv


In [72]:
df01 = pd.read_csv(r'C:\\Users\tinag\AppData\Local\Temp\Journal_Report_VZ\counter5_tr_j1_2021-01-2021-12_2000150893.csv', on_bad_lines='skip')
df01 = df.drop(df.index[0:11])
df.to_csv("report_stammdaten_cleaned")

df01 = pd.read_csv(r"C:\Users\tinag\AppData\Local\Temp\Journal_Report_VZ\counter5_tr_j1_2021-01-2021-12_2000150893.csv", sep=",", on_bad_lines='skip', header=None)
print(df01)

# csv Datei bereinigen, da standardmäßiger Header mit Stammdaten im COUNTER Format bei der Weiterarbeit stört (= Zeile 0 bis 11)
# COUNTER Daten beginnen erst in Zeile 15
# der Stammdaten-Header (= Zeile 0 bis 11) soll als Spalten in die Tabelle integriert werden
# Ausgabe des Stammdaten-Header

                    0                                                  1
0         Report_Name               Journal Requests (Excluding OA_Gold)
1           Report_ID                                              TR_J1
2             Release                                                  5
3    Institution_Name                        TH Bibliothek Aschaffenburg
4      Institution_ID                                      SN:2000150893
5        Metric_Types          Total_Item_Requests; Unique_Item_Requests
6      Report_Filters  Access_Method=Regular; Access_Type=Controlled;...
7   Report_Attributes                                                NaN
8          Exceptions                                                NaN
9    Reporting_Period         Begin_Date=2021-01-01; End_Date=2021-12-31
10            Created                               2022-07-01T09:27:29Z
11         Created_By                                     SpringerNature


In [73]:
print(df01.T)
# Zeilen der Spalte 0 zu Spalten transformieren

                                     0          1        2   \
0                           Report_Name  Report_ID  Release   
1  Journal Requests (Excluding OA_Gold)      TR_J1        5   

                            3               4   \
0             Institution_Name  Institution_ID   
1  TH Bibliothek Aschaffenburg   SN:2000150893   

                                          5   \
0                               Metric_Types   
1  Total_Item_Requests; Unique_Item_Requests   

                                                  6                  7   \
0                                     Report_Filters  Report_Attributes   
1  Access_Method=Regular; Access_Type=Controlled;...                NaN   

           8                                           9   \
0  Exceptions                            Reporting_Period   
1         NaN  Begin_Date=2021-01-01; End_Date=2021-12-31   

                     10              11  
0               Created      Created_By  
1  2022-07-01T09:27:

In [85]:
df02 = pd.read_csv(r'C:\\Users\tinag\AppData\Local\Temp\Journal_Report_VZ\counter5_tr_j1_2021-01-2021-12_2000150893.csv', sep=",")
df02 = df.drop(df.index[18:30])
df.to_csv("counter")

df02 = pd.read_csv(r"C:\Users\tinag\AppData\Local\Temp\Journal_Report_VZ\counter5_tr_j1_2021-01-2021-12_2000150893.csv")
print(df02)

# csv Datei bereinigen, da standardmäßiger Header mit Stammdaten im COUNTER Format bei der Weiterarbeit stört (= Zeile 0 bis 11)
# COUNTER Daten beginnen erst in Zeile 15
# Ausgabe der COUNTER Daten ab Zeile 15

# Bereinigung der Daten, da Fehler "Error tokenizing data. C error" auftritt
# Vermutung Fehler: Informationen (z. B. Buchtitel) enthalten ebenfalls Kommas, sodass ein Trennzeichenfehler auftritt

ParserError: Error tokenizing data. C error: Expected 2 fields in line 14, saw 23


In [60]:
data = {
    "Anbieter":[],
    "Report_Name":[],
    "Report_ID":[],
    "Metric Type":[],
    "Begin_Date":[],
    "End_Date":[],
    "Reporting_Period_Total":[],
    "Januar":[],
    "Februar":[],
    "März":[],
    "April":[],
    "Mai":[],
    "Juni":[],
    "Juli":[],
    "August":[],
    "September":[],
    "Oktober":[],
    "November":[],
    "Dezember":[],
}
df_csv = pd.DataFrame(data)
df_csv
# Dateframe "Ergebniss_Journal_Report" erstellen

,Anbieter,Report_Name,Report_ID,Metric Type,Begin_Date,End_Date,Reporting_Period_Total,Januar,Februar,März,April,Mai,Juni,Juli,August,September,Oktober,November,Dezember


In [61]:
df_csv.to_csv(path_or_buf= None,
                 sep= ";",
                 na_rep= "",
                 float_format= None,
                 columns= None,
                 header= True,
                 index= True,
                 index_label= None,
                 mode= "w",
                 encoding= None,
                 compression= "infer",
                 quoting= None,
                 quotechar= "'",
                 line_terminator= None,
                 chunksize= None,
                 date_format= None,
                 doublequote= True,
                 escapechar= None,
                 decimal= ",")

# Uwandlung DataFrame "Ergebniss_Journal_Report" in csv-Dateiformat

';Anbieter;Report_Name;Report_ID;Metric Type;Begin_Date;End_Date;Reporting_Period_Total;Januar;Februar;März;April;Mai;Juni;Juli;August;September;Oktober;November;Dezember\r\n'